In [1]:
import re
import requests

from tradeskill_reader import Item, Recipe, Misc
from tqdm.notebook import tqdm
from json import loads

### The full pipeline is as follows:
1. We find wowhead's list of all recipes.
2. We scrape the recipes and save them in a list. Because the page includes all `cooking spells`, we have to filter out the ones that are not recipes.
3. We create a Recipe object with information on the recipe id, spell id, name and reagents.

4. Then we create a list of all reagent ids that are used in the recipes.
5. We scrape wowhead for the name of each reagent.
6. And additionally find the mobs that drop each reagent. We specifically filter out the mobs that have a drop chance of less than 10%.
7. We save the reagent id, name and the mobs that drop it in a dictionary.

8. Then, for each recipe, we scrape wowhead for its page.
9. We find the buff that the recipe gives and save it in a file.
10. This buff is then inputted into the Recipe object.

11. We crate a LUA file that contains all the recipes and the relevant information.
12. We also create a LUA file that contains all the reagents and the relevant information.

### Read the recipe database from wow-head.

In [2]:
profession_collection = "https://www.wowhead.com/classic/spells/secondary-skills/cooking#0+17+18+1"
# profession_collection = "https://www.wowhead.com/wotlk/spells/secondary-skills/cooking#0+17"

req = requests.get(profession_collection, 'html.parser')

spells = re.findall("var listviewspells = \[(.*?)\];", req.text)[0]
spells = re.findall("\{(.*?)\}", spells)

### Create unique lists of all recipes and ingredients used in the them.

In [3]:
reagent_list = []
recipe_list = []

for spell in spells:
    recipe = Recipe.read(spell)
    
    # Skip misc spells (Cooking Fire, etc.)
    if isinstance(recipe, Misc):
        continue
    
    recipe_list.append(Recipe.read(spell)) # Save the recipe.
    # Make a list of all ingredient ids that were found
    reagent_list.extend([reagent_id for reagent_id, _ in recipe.reagents])

# Then keep all of the unique ingredients.
reagentid_list = list(set(reagent_list))

print("Found {} recipes and {} unique ingredients.".format(len(recipe_list), len(reagentid_list)))

Found 81 recipes and 89 unique ingredients.


### Read reagent ids to find their names and which mobs drop them.

In [115]:
def find_host_mobs(item_page: int, critical_chance: float = 0.1):
    try:
        items_on_wowhad = re.findall("dropped-by(.*?)\n\}\);", item_page, re.DOTALL)[0]
    except IndexError:
        return
    
    # Only keep data: onwards
    data = re.findall("data:(.+)", items_on_wowhad)[0]
    
    valid_enemies = {}
    for enemy in loads(data[:-1]):
        if enemy["outof"] < 100: # Skip entries without enough data.
            continue
        
        drop_chance = enemy["count"]/enemy["outof"]
        mob_id = enemy["id"]
        if drop_chance < critical_chance:
            continue
            
        # Round the drop chance to 2 decimal places.
        drop_chance = float("{:.2f}".format(drop_chance))
        
        # Add the mob to the list of valid enemies as a dictionary item.
        valid_enemies[mob_id] = [enemy["name"], drop_chance]
    
    return valid_enemies

In [74]:
item_page = f"https://www.wowhead.com/classic/item={item_list[0].id}"
req = requests.get(item_page, 'html.parser')
# save the item on a temporary txt file.
with open("item.txt", "w") as f:
    f.write(req.text)

In [99]:
item_list = []

# We now scrape wowhead for the name of each ingredient and the mobs that drop it.
for reagent_id in tqdm(reagentid_list):
    item_page = f"https://www.wowhead.com/classic/item={reagent_id}"
    req = requests.get(item_page, 'html.parser')
    
    reagent_name = re.findall("<title>(.*?) - Item", req.text)[0]
    
    # A dictionary of the id of a mob that drops the item: the drop chance.
    host_mobs = find_host_mobs(req.text, critical_chance = 0.1)
    
    item_list.append(Item(reagent_id, reagent_name, host_mobs))
    

  0%|          | 0/89 [00:00<?, ?it/s]

In [100]:
unique_mobs = []

for item in item_list:
    if item.dropped_by is None:
        continue
    
    for mob_id, drop_chance in item.dropped_by.items():
        unique_mobs.append(mob_id)

unique_mobs = list(set(unique_mobs))

# Print how many unique mobs we found.
print("Found {} unique mobs.".format(len(unique_mobs)))

Found 671 unique mobs.


In [114]:
# Open a file to write the data to.
# It should mimic a LUA table whose keys are the item ids.
# The values are a dictionary of the item name and a dictionary of the mobs that drop it.
with open("CookingReagents.lua", "w") as f:
    f.write("local BitesCookBook.ReagentData = {\n")
    
    for item in item_list:
        f.write("\t[{}] = {{\n".format(item.id))
        f.write("\t\tname = \"{}\",\n".format(item.name))
        
        # Next to each id we should comment out the name of the mob.
        if item.dropped_by is not None:
            f.write("\t\tdropped_by = {\n")
            for mob_id, (mob_name, drop_chance) in item.dropped_by.items():
                f.write("\t\t\t[{}] = {}, -- {}\n".format(mob_id, drop_chance, item.dropped_by[mob_id][0]))
            f.write("\t\t},\n")
        
        f.write("\t},\n")
    f.write("}\n")
    f.close()

### Read the recipe ids to find their buffs.

In [118]:
# open a file that we will write the buff on.
with open("buffs.txt", 'w') as f:
    for recipe in tqdm(recipe_list):
        recipe_id = recipe.id
        
        recipe_page = f"https://www.wowhead.com/classic/item={recipe_id}"
        req = requests.get(recipe_page, 'html.parser')
        
        try:
            buff = re.findall("class=\"q2\">(.*?)</a>", req.text, re.DOTALL)[0]
            buff = re.findall(">.*", buff)[0][1:]
            # Delete occurences of &nbsp;
            buff = buff.replace("&nbsp;", "")
        except IndexError:
            print("Missing tooltip for recipe id: ", recipe_id)
            continue
        
        # Write the buff on a file.
        f.write(buff)
        f.write("\n")
    
    f.close()


  0%|          | 0/81 [00:00<?, ?it/s]

We finally go through each recipe and create a LUA dictionary that the addon will read from.

In [125]:
# Read each line in the buffs.txt file.
with open("buffs.txt", 'r') as f:
    for line in f.readlines():
        # If the line ends in 'while eating.', we skip it.
        if line.endswith("while eating.\n"):
            continue
        
        # We need to find if the line mentions a stats buff.
        buff = re.findall("and gain (.*) for", line)
        print(buff)

['2 Stamina and Spirit']
['4 Stamina and Spirit']
['4 Stamina and Spirit']
['4 Stamina and Spirit']
['4 Stamina and Spirit']
['6 Stamina and Spirit']
['6 Stamina and Spirit']
['2 Stamina and Spirit']
['4 Stamina and Spirit']
['4 Stamina and Spirit']
['6 Stamina and Spirit']
['6 Stamina and Spirit']
['6 Stamina and Spirit']
['6 Stamina and Spirit']
['6 Stamina and Spirit']
['6 Stamina and Spirit']
['8 Stamina and Spirit']
['8 Stamina and Spirit']
['8 Stamina and Spirit']
['2 Stamina and Spirit']
[]
['2 Stamina and Spirit']
['4 Stamina and Spirit']
['4 Stamina and Spirit']
['6 Stamina and Spirit']
['6 Stamina and Spirit']
['4 Stamina and Spirit']
['6 Stamina and Spirit']
['8 Stamina and Spirit']
[]
['2 Stamina and Spirit']
[]
[]
['6 Stamina and Spirit']
['8 Stamina and Spirit']
['8 Stamina and Spirit']
['8 Stamina and Spirit']
['8 Stamina and Spirit']
['8 Stamina and Spirit']
[]
['12 Stamina and Spirit']
['12 Stamina and Spirit']
['12 Stamina and Spirit']
['2 Stamina and Spirit']
[]
[]
[

In [9]:
with open("CookingRecipes.lua", 'w') as f:
    f.write("local recipes = {\n")
    
    for i, spell in enumerate(spells):
        spell = Recipe.read(spell)
        
        # Actions and profession upgrades are also saved here.
        if type(spell) is Recipe:
            f.write(spell.get_LUA_list(id_dict))
    
    f.write("}\n\n")
    f.write("BitesCookBook_Recipes = recipes")
    f.close()

# Find the buffs of the spells

In [ ]:
# Some will say 'If you spend at least'...
# While others may say 'Also increases your'
# Others say 'Also restores'
# there is no rule of thumb for why this happens. We just have to find a general solution to solve for stat increases and mana/health gen.
# Some items have unique buffs too. Do we we include them, if so how?

# A table of capitalized abbreviations for the stats.
# INT = Intelligence
# STR = Strength
# AGI = Agility
# STA = Stamina
# SPI = Spirit

In [53]:
# Write req.text on a temporary file.
with open("wowhead_archive/recipe_test.html", 'w') as f:
    f.write(req.text)
    f.close()

In [46]:
# match the part that starts from g_items[2679].tooltip_enus until the end of line.
match = re.findall("Restores (.*?)<", req.text, re.DOTALL)[0]
match

'61.2 health over 18 sec. &nbsp;Must remain seated while eating. &nbsp;If you spend at least 10 seconds eating you will become well fed and gain 2 Stamina and Spirit for 15 min.'